In [ ]:
!pip install unsloth

ERROR: Could not find a version that satisfies the requirement google.colab (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for google.colab


In [ ]:
from google.colab import userdata
from huggingface_hub import login

login(userdata.get('HF_TOKEN'))
token = userdata.get('HF_TOKEN')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 2048

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it",
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    token = token,
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,  # rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from datasets import load_dataset

ds = load_dataset("gretelai/synthetic_text_to_sql", token=token)
ds = ds.shuffle()
train_ds = ds["train"].select(range(100000))

In [ ]:
user_prompt = """Given the <USER_QUERY> and the <SCHEMA>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<SCHEMA>
{context}
</SCHEMA>

<USER_QUERY>
{question}
</USER_QUERY>

<RESPONSE>
{response}
</RESPONSE>
"""

def format_prompts_for_training(examples):
    return {"prompt": user_prompt.format(question=examples["sql_prompt"], context=examples["sql_context"], response=examples["sql"])}

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "prompt",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-5,
        logging_steps = 100,
        optim="adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir="./model/gemma-270m-Text2SQL-Fine-tuned",
        report_to="wandb",
        run_name="Gemma-Text2SQL"
    ),
)

In [ ]:
import wandb
wandb.login()

In [ ]:
%env WANDB_PROJECT=Gemma-Text2SQL

In [ ]:
trainer_stats = trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
model.push_to_hub_merged("leotod/gemma-270m-Text2SQL-Fine-tuned", tokenizer, save_method = "merged_16bit", token = token)